In [1]:
# Import required libraries
import pandas as pd
import dash
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df.drop('Unnamed: 0',axis = 1,inplace = True)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
spacex_df.shape

(56, 6)

In [5]:
[min_payload, max_payload]

[0.0, 9600.0]

In [6]:
# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             options = [{'label':'All Sites', 'value':'ALL'},
                                                        {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                        {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                                        {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                                        {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                                       ],
                                             value = 'ALL',
                                             placeholder = 'Select a Launch Site here',
                                             searchable = True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id ='payload-slider',
                                                min = 0, max = 10000, step = 1000,
                                                value = [min_payload, max_payload]
                                               ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(value):
    filtered_df = spacex_df
    if value == 'ALL':
        fig = px.pie(filtered_df, values='class', 
                     names=filtered_df['Launch Site'], 
                     title='Total Success Launches by Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site']==value]
        fig = px.pie(filtered_df, 
                     names=filtered_df['class'],
                     title=f'Total Success Launches by site {value}')
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='payload-slider', component_property='value'),
              Input(component_id='site-dropdown', component_property='value')
             )

def get_scatter_plot(payload_value, dropdown_value):
    
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload_value[0],payload_value[1])]
    if dropdown_value == 'ALL':
        fig = px.scatter(filtered_df, x = 'Payload Mass (kg)', y = 'class',
                         color = 'Booster Version Category',
                         title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site']==dropdown_value]
        filtered_df = filtered_df[filtered_df['Payload Mass (kg)'].between(payload_value[0],payload_value[1])]
        fig = px.scatter(filtered_df, x = 'Payload Mass (kg)', y = 'class',
                         color = 'Booster Version Category',
                         title=f'Correlation between Payload and Success for site {dropdown_value}')
        return fig

# Run the app
#if __name__ == '__main__':
app.run_server(mode='inline')